# FE621 HW4

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import math

## Problem 1

### Question (a)

In [21]:
s0 = 100
k = 100
t = 1
r = 0.06
sigma = 0.2
div = 0.03

N = 300
M = 1000000

def MC_Euro(s0, k, t, r, sigma, div, N, M, type):
    dt = t / N
    nudt = (r - div - 0.5 * pow(sigma, 2)) * dt
    sigsdt = sigma * np.sqrt(dt)
    lnS = np.log(s0)
    
    lnSt = np.zeros(M)
    CT = np.zeros(M)
    
    for j in range(0, M):
        lnSt[j] = lnS
    for i in range(1, N + 1):
        epsilon = np.random.normal(0, 1, M)
        lnSt = lnSt + nudt + sigsdt * epsilon
    
    ST = np.exp(lnSt)
    for j in range(0, M):
        if type == 'call':
            CT[j] = max(0, ST[j] - k)
        else:
            CT[j] = max(0, k - ST[j])
            
    sum_CT = np.sum(CT)
    sum_CT2 = np.sum(CT + CT)
    
    value = sum_CT / M * np.exp(-r * t)
    SD = np.sqrt(abs(sum_CT2 - sum_CT * sum_CT / M) * np.exp(-2 * r * t) / (M - 1))
    SE = SD / np.sqrt(M)
    
    df = pd.DataFrame(columns = ['MC'])
    df['MC'] = [value, SD, SE]
    df.index = ['Option Value', 'Standard Deviation', 'Standard Error']
    
    return df

In [22]:
import time
# Call
start = time.clock()
Call = MC_Euro(s0, k, t, r, sigma, div, N, M, 'call')
end = time.clock()
time_call = end - start

# Put
start = time.clock()
Put = MC_Euro(s0, k, t, r, sigma, div, N, M, 'put')
end = time.clock()
time_put = end - start

df_MC = pd.concat([Call, Put], axis = 1)
df_MC.columns = ['MC_Call', 'MC_Put']
df_MC.loc['Time'] = [time_call, time_put]
df_MC

,MC_Call,MC_Put
Option Value,9.156270,6.262979
Standard Deviation,8.160346,5.237216
Standard Error,0.008160,0.005237
Time,9.458458,9.580851


### Question (b)

In [23]:
def MC_AntitheticVariates(s0, k, t, r, sigma, div, N, M, type):
    dt = t/N
    nudt = (r-div-0.5*pow(sigma,2))*dt
    sigsdt = sigma*np.sqrt(dt)
    
    lnS = np.log(s0)
    lnSt1 = np.zeros(M)
    lnSt2 = np.zeros(M)
    CT = np.zeros(M)
    
    for j in range(0, M):
        lnSt1[j] = lnS
        lnSt2[j] = lnS
    for i in range(1,N+1):
        epsilon = np.random.normal(0,1,M)
        lnSt1 = lnSt1 + nudt + sigsdt*epsilon
        lnSt2 = lnSt2 + nudt + sigsdt*(-epsilon)
    ST1 = np.exp(lnSt1)
    ST2 = np.exp(lnSt2)
    
    for j in range(0, M):
        if type=='call':
            CT[j] = 0.5*(max(0, ST1[j] - k) + max(0, ST2[j] - k))
        else:
            CT[j] = 0.5*(max(0, k - ST1[j]) + max(0, k - ST2[j]))

    sum_CT = np.sum(CT)
    sum_CT2 = np.sum(CT*CT)
    
    value = sum_CT/M*np.exp(-r*t)
    SD = np.sqrt((sum_CT2 - sum_CT*sum_CT/M)*np.exp(-2*r*t)/(M-1))
    SE = SD/np.sqrt(M)
    df = pd.DataFrame(columns=['MC'])
    df['MC'] = [value, SD, SE]
    df.index = ['Option Value', 'Standard Deviation', 'Standard Error']
    return df

In [24]:
import time
# Call
start =time.clock()
Call = MC_AntitheticVariates(s0, k, t, r, sigma, div, N, M, 'call')
end = time.clock()

# Put
start =time.clock()
Put = MC_AntitheticVariates(s0, k, t, r, sigma, div, N, M, 'put')
end = time.clock()
time_put = end - start

df_AV = pd.concat([Call, Put],axis=1)
df_AV.columns = ['AntitheticV_Call', 'AntitheticV_Put']
df_AV.loc['Time'] = [time_call, time_put]
df_AV

,AntitheticV_Call,AntitheticV_Put
Option Value,9.131144,6.272302
Standard Deviation,7.204330,4.638993
Standard Error,0.007204,0.004639
Time,9.458458,12.574296


In [25]:
def Black_Scholes_Delta(s0, k, tt, t, r, sigma, div, type):
    d1 = (np.log(s0/k)+(r+pow(sigma,2)/2-div)*t)/(sigma*np.sqrt(t))
    
    if type=="call":
        delta = np.exp(-div*tt)*stats.norm.cdf(d1)
        return delta
    elif type=="put":
        delta = np.exp(-div*tt)*stats.norm.cdf(d1) - 1
        return delta

In [28]:
def MC_Delta(s0, k, t, r, sigma, div, N, M, type):
    dt = t/N
    nudt = (r-div-0.5*pow(sigma,2))*dt
    sigsdt = sigma*np.sqrt(dt)
    erddt = np.exp((r-div)*dt)
    beta1 = -1
    
    St = np.zeros(M)
    cv = np.zeros(M)
    CT = np.zeros(M)
    for j in range(0, M):
        St[j] = s0
        cv[j] = 0

    for i in range(1,N+1):
        tt = (i-1)*dt
        delta = Black_Scholes_Delta(St, k, tt, t, r, sigma, div, type)
        epsilon = np.random.normal(0,1,M)
        Stn = St*np.exp(nudt + sigsdt*epsilon)
        cv = cv + delta*(Stn - St*erddt)
        St = Stn

    for j in range(0, M):
        if type=='call':
            CT[j] = max(0, St[j] - k) + beta1*cv[j]
        else:
            CT[j] = max(0, k - St[j]) + beta1*cv[j]

    sum_CT = np.sum(CT)
    sum_CT2 = np.sum(CT*CT)
        
    value = sum_CT/M*np.exp(-r*t)
    SD = np.sqrt((sum_CT2 - sum_CT*sum_CT/M)*np.exp(-2*r*t)/(M-1))
    SE = SD/np.sqrt(M)

    df = pd.DataFrame(columns=['MC_Delta'])
    df['MC_Delta'] = [value, SD, SE]
    df.index = ['Option Value', 'Standard Deviation', 'Standard Error']

    return df

In [29]:
import time
# Call
start =time.clock()
Call = MC_Delta(s0, k, t, r, sigma, div, N, M, 'call')
end = time.clock()
time_call = end - start

# Put
start =time.clock()
Put = MC_Delta(s0, k, t, r, sigma, div, N, M, 'put')
end = time.clock()
time_put = end - start

df_DB = pd.concat([Call, Put],axis=1)
df_DB.columns = ['DeltaBased_Call', 'DeltaBased_Put']
df_DB.loc['Time'] = [time_call, time_put]
df_DB

,DeltaBased_Call,DeltaBased_Put
Option Value,9.136999,6.266909
Standard Deviation,2.043176,2.044053
Standard Error,0.002043,0.002044
Time,31.556437,31.903984


In [30]:
def MC_AntitheticDelta(s0, k, t, r, sigma, div, N, M, type):
    dt = t/N
    nudt = (r-div-0.5*pow(sigma,2))*dt
    sigsdt = sigma*np.sqrt(dt)
    erddt = np.exp((r-div)*dt)
    beta1 = -1

    St1 = np.zeros(M)
    St2 = np.zeros(M)
    cv1 = np.zeros(M)
    cv2 = np.zeros(M)
    CT = np.zeros(M)

    for j in range(0, M):
        St1[j] = s0
        St2[j] = s0
        cv1[j] = 0
        cv2[j] = 0

    for i in range(1,N+1):
        tt = (i-1)*dt
        delta1 = Black_Scholes_Delta(St1, k, tt, t, r, sigma, div, type)
        delta2 = Black_Scholes_Delta(St2, k, tt, t, r, sigma, div, type)
        epsilon = np.random.normal(0,1,M)
        Stn1 = St1*np.exp(nudt + sigsdt*epsilon)
        Stn2 = St2*np.exp(nudt + sigsdt*(-epsilon))
        cv1 = cv1 + delta1*(Stn1 - St1*erddt)
        cv2 = cv2 + delta2*(Stn2 - St2*erddt)
        St1 = Stn1
        St2 = Stn2

    for j in range(0, M):
        if type=='call':
            CT[j] = 0.5*(max(0, St1[j] - k) + beta1*cv1[j] +max(0, St2[j] - k) + beta1*cv2[j])
        else:
            CT[j] = 0.5*(max(0, k - St1[j]) + beta1*cv1[j] +max(0, k - St2[j]) + beta1*cv2[j])

    sum_CT = np.sum(CT)
    sum_CT2 = np.sum(CT*CT)
    value = sum_CT/M*np.exp(-r*t)
    SD = np.sqrt((sum_CT2 - sum_CT*sum_CT/M)*np.exp(-2*r*t)/(M-1))
    SE = SD/np.sqrt(M)
    df = pd.DataFrame(columns=['MC_AD'])
    df['MC_AD'] = [value, SD, SE]
    df.index = ['Option Value', 'Standard Deviation', 'Standard Error']
    return df

In [31]:
import time
# Call
start =time.clock()
Call = MC_AntitheticDelta(s0, k, t, r, sigma, div, N, M, 'call')
end = time.clock()
time_call = end - start

# Put
start =time.clock()
Put = MC_AntitheticDelta(s0, k, t, r, sigma, div, N, M, 'put')
end = time.clock()
time_put = end - start

df_AD = pd.concat([Call, Put],axis=1)
df_AD.columns = ['AntitheticDelta_Call', 'AntitheticDelta_Put']
df_AD.loc['Time'] = [time_call, time_put]
df_AD

,AntitheticDelta_Call,AntitheticDelta_Put
Option Value,9.136380,6.263931
Standard Deviation,2.021024,2.006186
Standard Error,0.002021,0.002006
Time,57.419999,57.204284


In [32]:
df = pd.concat([df_MC, df_AV, df_DB, df_AD],axis=1)
df

,MC_Call,MC_Put,AntitheticV_Call,AntitheticV_Put,DeltaBased_Call,DeltaBased_Put,AntitheticDelta_Call,AntitheticDelta_Put
Option Value,9.156270,6.262979,9.131144,6.272302,9.136999,6.266909,9.136380,6.263931
Standard Deviation,8.160346,5.237216,7.204330,4.638993,2.043176,2.044053,2.021024,2.006186
Standard Error,0.008160,0.005237,0.007204,0.004639,0.002043,0.002044,0.002021,0.002006
Time,9.458458,9.580851,9.458458,12.574296,31.556437,31.903984,57.419999,57.204284


From the table we can see that the European option prices calculated by four methods are almost same and the SD and SE are decreasingn but the running time are increasing. Among them,  using both Antithetic Variates and Delta-based Control Variates method produces the least SD and SE and the cost the most time.

## Problem 2

In [33]:
s0 = 100
v0 = 0.010201
c0 = 6.8061
k = 100
t = 1

kappa = 6.21
theta = 0.019
omiga = 0.61
rho = -0.7

alpha = 0.5
beta = 1

N = 300
M = 1000000

def Same(x):
    return x

def Max(x):
    x[x < 0] = 0
    return x

def Abs(x):
    return abs(x)

In [34]:
def Euler_Scheme(s0, v0, k, t, r, kappa, theta, omiga, rho, alpha, beta, N, M, type, f1, f2, f3):
    dt = t/N
    St = np.zeros(M)
    CT = np.zeros(M)
    vt = np.zeros(M)

    for j in range(0, M):
        St[j] = s0
        vt[j] = v0

    for i in range(1,N+1):
        z1 = np.random.normal(0,1,M)
        z2 = np.random.normal(0,1,M)
        zs = rho*z1 + np.sqrt(1-pow(rho,2))*z2
        vtn = f3(f1(vt) - kappa*dt*(f2(vt)-theta)+ omiga*pow(f3(vt),alpha)*np.sqrt(dt)*z1)
        St = St*np.exp((r-0.5*vt)*dt+np.sqrt(vt*dt)*zs)
        vt = vtn
    
    for j in range(0, M):
        if type=='call':
            CT[j] = max(0, St[j] - k)
        else:
            CT[j] = max(0, k - St[j])

    sum_CT = np.sum(CT)
    sum_CT2 = np.sum(CT*CT)
    value = sum_CT/M*np.exp(-r*t)
    bias = value - c0
    var = (sum_CT2 - sum_CT*sum_CT/M)*np.exp(-2*r*t)/(M-1)
    RMSE = np.sqrt(pow(bias,2)+var)
    
    df = pd.DataFrame(columns=['Euler'])
    df['Euler'] = [value, bias, RMSE]
    df.index = ['Option Value', 'Bias', 'RMSE']
    
    return df

In [35]:
type = 'call'
import time
start =time.clock()
df_Absorption = Euler_Scheme(s0, v0, k, t, r, kappa, theta, omiga, 
                             rho, alpha, beta, N, M, type, Max, Max, Max)
end = time.clock()
time_call = end - start

df_Absorption.columns = ['Absorption']
df_Absorption.loc['Time'] = [time_call]
df_Absorption

,Absorption
Option Value,8.715586
Bias,1.909486
RMSE,8.642858
Time,26.200527


In [36]:
start =time.clock()
df_Reflection = Euler_Scheme(s0, v0, k, t, r, kappa, theta, omiga, 
                             rho, alpha, beta, N, M, type, Abs, Abs, Abs)
end = time.clock()
time_call = end - start
df_Reflection.columns = ['Reflection']
df_Reflection.loc['Time'] = [time_call]
df_Reflection

,Reflection
Option Value,8.770151
Bias,1.964051
RMSE,8.810494
Time,27.810435


In [37]:
start =time.clock()
df_HighamMao = Euler_Scheme(s0, v0, k, t, r, kappa, theta, omiga, 
                            rho, alpha, beta, N, M, type, Same, Same, Abs)
end = time.clock()
time_call = end - start

df_HighamMao.columns = ['Higham_And_Mao']
df_HighamMao.loc['Time'] = [time_call]
df_HighamMao

,Higham_And_Mao
Option Value,8.773467
Bias,1.967367
RMSE,8.807786
Time,27.542667


In [38]:
start =time.clock()
df_Partial = Euler_Scheme(s0, v0, k, t, r, kappa, theta, omiga, 
                          rho, alpha, beta, N, M, type, Same, Same, Max)
end = time.clock()
time_call = end - start

df_Partial.columns = ['Partial_Truncation']
df_Partial.loc['Time'] = [time_call]
df_Partial

,Partial_Truncation
Option Value,8.722462
Bias,1.916362
RMSE,8.649698
Time,26.691118


In [39]:
start =time.clock()
df_Full = Euler_Scheme(s0, v0, k, t, r, kappa, theta, omiga, 
                       rho, alpha, beta, N, M, type, Same, Max, Max)
end = time.clock()
time_call = end - start

df_Full.columns = ['Full_Truncation']
df_Full.loc['Time'] = [time_call]
df_Full

,Full_Truncation
Option Value,8.685541
Bias,1.879441
RMSE,8.615665
Time,26.549863


In [40]:
df_Euler_Schemes = pd.concat([df_Absorption, df_Reflection, 
                              df_HighamMao, df_Partial, df_Full],axis=1)
df_Euler_Schemes

,Absorption,Reflection,Higham_And_Mao,Partial_Truncation,Full_Truncation
Option Value,8.715586,8.770151,8.773467,8.722462,8.685541
Bias,1.909486,1.964051,1.967367,1.916362,1.879441
RMSE,8.642858,8.810494,8.807786,8.649698,8.615665
Time,26.200527,27.810435,27.542667,26.691118,26.549863


## Problem 3

### Question 1

In [41]:
A = 10000000 # total amount of dollars
x0 = 80
z0 = 6.1
A_x = A * 0.4
N_x = A_x / x0 # Number of IBM shares
A_z = A * 0.3
N_z = A_z * z0

df = pd.DataFrame(columns=['Number of Shares', 'Amount in Yuan'])
df.loc[0] = [N_x, N_z]
df

,Number of Shares,Amount in Yuan
0,50000.0,18300000.0


### Question 2 & 3

In [42]:
x0 = 80
y0 = 90000
z0 = 6.1
t = 10 / 252
dt = 0.001

alpha = 0.01
N = 10
M = 3000000

def Multiple_Monte_Carlo(x0, y0, z0, dt, alpha, N, M):
    t = N/252
    x = np.zeros(M)
    y = np.zeros(M)
    z = np.zeros(M)
    
    for j in range(0, M):
        x[j] = x0
        y[j] = y0
        z[j] = z0

    for i in range(1,round(t/dt+1)):
        epsilon1 = np.random.normal(0,1,M)
        epsilon2 = np.random.normal(0,1,M)
        epsilon3 = np.random.normal(0,1,M)
        x = x + 0.01*x*dt + 0.3*x*epsilon1*np.sqrt(dt)
        y = y + 100*(90000 + 1000*t - y)*dt + np.sqrt(y)*epsilon2*np.sqrt(dt)
        z = z + 5*(6-z)*dt + 0.01*np.sqrt(z)*epsilon3*np.sqrt(dt)
    A_y = A*0.3
    N_y = A_y/y0
    
    p = N_x*x + N_y*y + N_z/z # portfolio value
    R = (p - A)/A # calculate return
    R.sort()
    
    n = math.floor(alpha*M)
    VAR = R[n-1] # VAR value
    CVAR = sum(R[:n-1])/n # CVAR value
    
    df = pd.DataFrame(columns=['Multiple_MC'])
    df['Multiple_MC'] = [VAR, CVAR]
    df.index = ['VAR','CVAR']

    return df

In [43]:
Multiple_Monte_Carlo(x0, y0, z0, dt, alpha, N, M)

,Multiple_MC
VAR,-0.051699
CVAR,-0.058617
